In [1]:
import numpy as np
import mujoco as mj
from mujoco.glfw import glfw # MuJoCo 시뮬레이터 GLFW (OpenGL 기반 윈도우/입력 라이브러리)

import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from utils.mouse_callbacks import MouseCallbacks # 마우스 이벤트(카메라 회전/이동/줌 등) 콜백
from utils.keyboard_callbacks import KeyboardCallbacks # 키보드 입력(로봇 제어 및 초기화) 콜백 
from utils.scene_creator import SceneCreator # MJCF xml 파일 조합 및 scene 빌드용 헬퍼

In [2]:
cwd = os.getcwd() 
PROJECT_ROOT = os.path.dirname(cwd)
print(f"project root: {PROJECT_ROOT}")

assets_dir = os.path.join(PROJECT_ROOT, "assets")
base_env_path = os.path.join(assets_dir, "scenes", "floor_sky.xml")   # 환경 MJCF
robot_path = os.path.join(assets_dir, "robots", "pinky", "pinky.xml") # 로봇 MJCF

# 배치할 오브젝트 정보
objects_to_spawn = [
    {"path": os.path.join(assets_dir, "objects", "can.xml"), "name": "can_1",  "pos": "0.5 0.3 0.05"},
    {"path": os.path.join(assets_dir, "objects", "milk.xml"), "name": "milk_1", "pos": "0.5 0.0 0.05"},
    {"path": os.path.join(assets_dir, "objects", "lemon.xml"), "name": "lemon_1","pos": "0.5 -0.3 0.05"},
]

# 각 MJCF들을 병합
scene_xml_string = SceneCreator.build_mjcf_scene(
    base_env_path=base_env_path,
    robot_path=robot_path,
    objects_to_spawn=objects_to_spawn,
    save_xml=True
)

project root: c:\Users\addinedu\short\pinky_mujoco_menagerie
saved scene.xml


In [3]:
# XML 문자열로부터 MuJoCo 모델(물리/구조 정보) 생성
model = mj.MjModel.from_xml_string(scene_xml_string)
# 모델에 대한 시뮬레이션 데이터(상태 변수, 센서값 등) 생성
data = mj.MjData(model)

# mujoco 카메라(메인, 로봇 시점), 옵션, 씬(각 시점의 3D 그래픽 정보) 객체 선언
main_cam = mj.MjvCamera()
robot_cam = mj.MjvCamera()
opt = mj.MjvOption()
scene_main = mj.MjvScene(model, maxgeom=10000)
scene_robot = mj.MjvScene(model, maxgeom=10000)

In [4]:
# GLFW 초기화 및 윈도우 생성
glfw.init()
window_main = glfw.create_window(900, 900, "Main View", None, None)
window_robot = glfw.create_window(640, 480, "Camera View", None, window_main)

# OpenGL 컨텍스트 연결 및 렌더 컨텍스트 생성
glfw.make_context_current(window_main)
ctx_main = mj.MjrContext(model, mj.mjtFontScale.mjFONTSCALE_150.value)
glfw.make_context_current(window_robot)
ctx_robot = mj.MjrContext(model, mj.mjtFontScale.mjFONTSCALE_150.value)

# 수직 동기화 설정: 1 활성화, 0 비활성화
glfw.swap_interval(1)
# Free Camera, 렌더링 옵션 기본값
mj.mjv_defaultFreeCamera(model, main_cam)
mj.mjv_defaultOption(opt)

In [ ]:
# 메인 카메라: 자유 이동(Free) 모드 설정
main_cam.type = mj.mjtCamera.mjCAMERA_FREE
main_cam.azimuth = 0     # 카메라 수평(좌우) 회전 각도
main_cam.elevation = -30 # 카메라 수직(상하) 회전 각도
main_cam.distance = 2    # 카메라와 대상(lookat) 사이 거리
main_cam.lookat = np.array([0.0, 0.0, 0.5]) # 카메라가 바라보는 좌표

# 로봇 카메라: 고정(Fixed) 모드 및 특정 카메라 ID로 지정
robot_cam.type = mj.mjtCamera.mjCAMERA_FIXED 
cam_id = mj.mj_name2id(model, mj.mjtObj.mjOBJ_CAMERA, "camera")
robot_cam.fixedcamid = cam_id

# 메인 씬: 그림자/반사 효과 비활성화 (성능 향상 목적)
scene_main.flags[mj.mjtRndFlag.mjRND_SHADOW] = False
scene_main.flags[mj.mjtRndFlag.mjRND_REFLECTION] = False

In [6]:
# 수직 동기화 설정: 1 활성화, 0 비활성화
glfw.swap_interval(1)

# 콜백 핸들러 객체 생성
mousecallbacks = MouseCallbacks()
kbdcallbacks = KeyboardCallbacks()

# GLFW 콜백 등록 (키보드, 커서, 마우스 버튼, 마우스 스크롤)
glfw.set_key_callback(window_main, 
                    lambda w, k, sc, act, m: kbdcallbacks.keyboardGLFW(w, k, sc, act, m, model, data))
glfw.set_cursor_pos_callback(window_main, 
                    lambda w, x, y: mousecallbacks.mouse_move(w, x, y, model, scene_main, main_cam))
glfw.set_mouse_button_callback(window_main, 
                    lambda w, b, act, m: mousecallbacks.mouse_button(w, b, act, m))
glfw.set_scroll_callback(window_main, 
                    lambda w, xo, yo: mousecallbacks.scroll(w, xo, yo, model, scene_main, main_cam))

In [7]:
# 윈도우에 시뮬레이션과 오버레이 텍스트 렌더링
def render_window(window, scene, ctx, cam, model, data, opt, overlay_text=None):
    glfw.make_context_current(window)
    w, h = glfw.get_framebuffer_size(window)
    viewport = mj.MjrRect(0, 0, w, h)
    mj.mjv_updateScene(model, data, opt, None, cam, mj.mjtCatBit.mjCAT_ALL.value, scene)
    mj.mjr_render(viewport, scene, ctx)
    if overlay_text:
        mj.mjr_overlay(
            mj.mjtFont.mjFONT_NORMAL, 
            mj.mjtGridPos.mjGRID_TOPRIGHT, 
            viewport, overlay_text, None, ctx
        )
    glfw.swap_buffers(window)

In [8]:
# 시뮬레이션에서 imu data를 읽어 반환
def get_imu_data():
    qpos = data.qpos
    quat_mujoco = qpos[3:7]
    orientation = [quat_mujoco[1], quat_mujoco[2], quat_mujoco[3], quat_mujoco[0]]

    accel = data.sensordata[0:3]
    gyro  = data.sensordata[3:6]

    return (
        "Orientation (Quat)\n"
        f"  x: {orientation[0]:.2f}\n"
        f"  y: {orientation[1]:.2f}\n"
        f"  z: {orientation[2]:.2f}\n"
        f"  w: {orientation[3]:.2f}\n"
        "Gyro \n"
        f"  x: {gyro[0]:.2f}\n"
        f"  y: {gyro[1]:.2f}\n"
        f"  z: {gyro[2]:.2f}\n"
        "Acceleration\n"
        f"  x: {accel[0]:.2f}\n"
        f"  y: {accel[1]:.2f}\n"
        f"  z: {accel[2]:.2f}"
    )

In [9]:
while not glfw.window_should_close(window_main):
    time_prev = data.time
    while (data.time - time_prev < 1.0 / 60.0): # 60fps
        try:
            mj.mj_step(model, data)
        except mj.MjError as e:
            print(f"시뮬레이션 불안정성 감지: {e}. 데이터를 리셋합니다.")
            mj.mj_resetData(model, data)
            mj.mj_forward(model, data)
            break

    overlay_text = get_imu_data()
    render_window(window_main, scene_main, ctx_main, main_cam, model, data, opt, overlay_text)
    render_window(window_robot, scene_robot, ctx_robot, robot_cam, model, data, opt)

    glfw.poll_events()
glfw.terminate()